In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from pandas import json_normalize
from datetime import datetime
import pandas_ta as ta
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pycoingecko import CoinGeckoAPI
import time
from yattag import Doc
import requests
# https://medium.com/@chris_42047/how-to-build-a-momentum-screener-for-over-13k-crypto-coins-for-free-in-python-7f6901435c33

In [2]:
def get_coin_ids(uri="https://api.coingecko.com/api/v3/coins/list"):
    try:
        # uri = 'https://raw.githubusercontent.com/justmobiledev/python-algorithmic-trading/main/data/coingecko-list.json'
        response = requests.get(uri).json()
        return response
    except Exception as e:
        print('Failed to look up Coinbase assets', e)
        return None

In [3]:
# list of all coins but no stats
coins = pd.DataFrame(get_coin_ids("https://api.coingecko.com/api/v3/coins/list"))
coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0-knowledge-network,0kn,0 Knowledge Network
3,0-mee,ome,O-MEE
4,0vix-protocol,vix,0VIX Protocol
...,...,...,...
12852,zynecoin,zyn,Zynecoin
12853,zynergy,zyn,Zynergy
12854,zyrri,zyr,Zyrri
12855,zyx,zyx,ZYX


In [22]:
# stats only show 250 at a time
coin_stats = pd.DataFrame(get_coin_ids("https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&per_page=250"))
coin_stats

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,48394.000000,950470811287,1,1.017047e+12,2.117781e+10,48729.000000,...,2.100000e+07,21000000.0,69045.000,-29.76007,2021-11-10T14:24:11.849Z,67.810000,71419.99962,2013-07-06T00:00:00.000Z,None,2024-02-12T02:18:34.236Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2511.960000,301956884486,2,3.019569e+11,6.836377e+09,2535.790000,...,1.201700e+08,NaN,4878.260,-48.43878,2021-11-10T14:24:19.604Z,0.432979,580826.91536,2015-10-20T00:00:00.000Z,"{'times': 68.38047796130013, 'currency': 'btc'...",2024-02-12T02:18:38.395Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999574,96362495859,3,9.636250e+10,2.634052e+10,1.004000,...,9.647514e+10,NaN,1.320,-24.33603,2018-07-24T00:00:00.000Z,0.572521,74.85935,2015-03-02T00:00:00.000Z,None,2024-02-12T02:15:14.241Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,320.510000,49341759976,4,4.934176e+10,4.461405e+08,324.830000,...,1.538562e+08,200000000.0,686.310,-53.17416,2021-05-10T07:24:17.097Z,0.039818,807000.53569,2017-10-19T00:00:00.000Z,None,2024-02-12T02:18:34.069Z
4,solana,sol,Solana,https://assets.coingecko.com/coins/images/4128...,107.930000,47187626126,5,6.143809e+10,1.454562e+09,110.270000,...,5.693771e+08,NaN,259.960,-58.40885,2021-11-06T21:54:35.825Z,0.500801,21489.41500,2020-05-11T19:35:23.449Z,None,2024-02-12T02:18:35.450Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,ontology,ont,Ontology,https://assets.coingecko.com/coins/images/3447...,0.239175,215227175,246,2.390887e+08,5.400741e+06,0.240304,...,1.000000e+09,NaN,10.920,-97.79950,2018-05-03T01:43:05.844Z,0.145208,65.48962,2022-12-30T07:46:17.182Z,None,2024-02-12T02:18:40.047Z
246,ssv-network,ssv,SSV Network,https://assets.coingecko.com/coins/images/1915...,30.260000,214695623,247,3.416888e+08,1.462464e+07,30.970000,...,1.127687e+07,NaN,49.520,-38.65894,2023-02-27T23:15:10.196Z,3.670000,727.43909,2022-06-18T20:56:57.599Z,None,2024-02-12T02:18:37.147Z
247,tectum,tet,Tectum,https://assets.coingecko.com/coins/images/2957...,29.650000,213925088,248,2.952582e+08,1.846576e+06,29.790000,...,1.000000e+07,10000000.0,45.030,-34.40127,2024-01-07T05:40:48.338Z,3.050000,867.08673,2023-10-10T02:50:53.429Z,None,2024-02-12T02:18:19.046Z
248,zelcash,flux,Flux,https://assets.coingecko.com/coins/images/5163...,0.611933,209793866,249,2.698949e+08,6.767059e+06,0.621923,...,4.400000e+08,NaN,3.330,-81.51442,2022-01-03T10:18:02.680Z,0.016404,3647.95333,2021-01-13T05:29:57.987Z,None,2024-02-12T02:18:39.353Z


In [24]:
def get_coin_info(cg, coin_id):
    try:
        response = cg.get_coin_by_id(coin_id)
        categories = response['categories']
        public_notice = response['public_notice']
        name = response['name']
        description = response['description']['en']
        links = response['links']
        homepage_link = links['homepage']
        blockchain_site = links['blockchain_site']
        if blockchain_site is not None:
            blockchain_site = ",".join(blockchain_site)
        official_forum_url = links['official_forum_url']
        chat_url = links['chat_url']
        announcement_url = links['announcement_url']
        twitter_screen_name = links['twitter_screen_name']
        facebook_username = links['facebook_username']
        telegram_channel_identifier = links['telegram_channel_identifier']
        subreddit_url = links['subreddit_url']
        sentiment_votes_up_percentage = response['sentiment_votes_up_percentage']
        sentiment_votes_down_percentage = response['sentiment_votes_down_percentage']
        market_cap_rank = response['market_cap_rank']
        coingecko_rank = response['coingecko_rank']
        coingecko_score = response['coingecko_score']
        community_score = response['community_score']
        liquidity_score = response['liquidity_score']
        public_interest_score = response['public_interest_score']
        row = pd.DataFrame(
            {'id': [coin_id], 'name': [name], 'categories': [categories], 'public_notice': [public_notice], \
             'description': [description], 'homepage_link': [homepage_link], \
             'blockchain_site': [blockchain_site], 'official_forum_url': [official_forum_url], \
             'chat_url': [chat_url], 'announcement_url': [announcement_url], \
             'twitter_screen_name': [twitter_screen_name], 'facebook_username': [facebook_username], \
             'telegram_channel_identifier': [telegram_channel_identifier], 'subreddit_url': [subreddit_url], \
             'sentiment_votes_up_percentage': [sentiment_votes_up_percentage], \
             'sentiment_votes_down_percentage': [sentiment_votes_down_percentage], \
             'market_cap_rank': [market_cap_rank], \
             'coingecko_rank': [coingecko_rank], \
             'coingecko_score': [coingecko_score], 'coingecko_score': [coingecko_score], \
             'community_score': [community_score], 'liquidity_score': [liquidity_score], \
             'public_interest_score': [public_interest_score]})
        row = row.set_index('id')
        return row
    except Exception as e:
        print('Failed to fetch CoinGecko coin info', e)
        return None

In [26]:
def build_report(coin_ids, info_map):
    # doc, tag, text, line = Doc().tagtext()
    doc, tag, text, line = Doc().ttl()
    now_str = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    with tag('html'):
        with tag('head'):
            with tag('style'):
                text("""div{ width: 100%; }
                        html { margin: 0; padding: 10px; background: #1e81b0;}
                        body { font: 12px verdana, sans-serif;line-height: 1.88889; margin: 5%; background: #ffffff; padding: 1%; width: 90%; }
                        p { margin-top: 5px; text-align: justify; font: normal 0.9em verdana, sans-serif;color:#484848}
                        li { font: normal 0.8em verdana, sans-serif;color:#484848}
                        h1 { font: normal 1.8em verdana, sans-serif; letter-spacing: 1px; margin-bottom: 0; color: #063970,}
                        h2 { font: normal 1.6em verdana, sans-serif; letter-spacing: 1px; margin-bottom: 0; color: #154c79}
                        h3 { font: normal 1.6em verdana, sans-serif; letter-spacing: 1px; margin-bottom: 0; color: #154c79}
                        p.bold_text{ font: normal 0.9em verdana, sans-serif; letter-spacing: 1px; margin-bottom: 0; color: #154c79; font-weight: bold}""")
        with tag('body', id='body'):
            with tag('h1'):
                text(f"Trending Crypto Report for {now_str}")
            with tag('hr'):
                text('')
            for coin_id in coin_ids:
                coin_df = info_map[coin_id]
                with tag('h2'):
                    text(f"{coin_id.upper()}")
                with tag('div'):
                    line('p', f"Name: {coin_df['name'].values[0]}", klass="bold_text")
                    line('p', coin_df['name'].values[0])
                    line('p', f"Description:", klass="bold_text")
                    line('p', coin_df['description'].values[0])
                    line('p', f"Categories:", klass="bold_text")
                    category_list = ""
                    if coin_df['categories'].values[0] is not None:
                        for category_str in coin_df['categories'].values[0]:
                            if category_str is None:
                                continue
                            if len(category_list) > 0:
                                category_list += ", "
                            category_list += category_str
                    line('p', f"{category_list}")
                    line('p', f"Public Notice:", klass="bold_text")
                    line('p', f"{coin_df['public_notice'].values[0]}")
                    line('p', f"Links:", klass="bold_text")
                    with tag('ul', id='links-list'):
                        url_list = ""
                        if coin_df['homepage_link'].values[0] is not None:
                            for link_str in coin_df['homepage_link'].values[0]:
                                url_list += link_str
                        line('li', f"Home Page: {url_list}")
                        line('li', f"Blockchain Site: {coin_df['blockchain_site'].values[0]}")
                        url_list = ""
                        if coin_df['official_forum_url'].values[0] is not None:
                            for link_str in coin_df['official_forum_url'].values[0]:
                                url_list += link_str
                        line('li', f"Official Forum URLs: {url_list}")
                        url_list = ""
                        if coin_df['chat_url'].values[0] is not None:
                            for link_str in coin_df['chat_url'].values[0]:
                                url_list += link_str
                        line('li', f"Chat URLs: {url_list}")
                    line('p', f"Social Media:", klass="bold_text")
                    with tag('ul', id='social-list'):
                        line('li', f"Twitter: {coin_df['twitter_screen_name'].values[0]}")
                        line('li', f"Facebook: {coin_df['facebook_username'].values[0]}")
                        line('li', f"Telegram: {coin_df['telegram_channel_identifier'].values[0]}")
                    line('p', f"Sentiment:", klass="bold_text")
                    with tag('ul', id='sentiment-list'):
                        line('li', f"Votes Up: {coin_df['sentiment_votes_up_percentage'].values[0]}")
                        line('li', f"Votes Down: {coin_df['sentiment_votes_down_percentage'].values[0]}")
                    line('p', f"Ranks:", klass="bold_text")
                    with tag('ul', id='sentiment-list'):
                        line('li', f"Market Cap Rank: {coin_df['market_cap_rank'].values[0]}")
                        line('li', f"Gecko Rank: {coin_df['coingecko_rank'].values[0]}")
                        line('li', f"Gecko Score: {coin_df['coingecko_score'].values[0]}")
                        line('li', f"Community Score: {coin_df['community_score'].values[0]}")
                        line('li', f"Public Interest Score: {coin_df['public_interest_score'].values[0]}")
                    with tag('div', id='photo-container'):
                        line('p', f"Day Plot:", klass="bold_text")
                        doc.stag('img', src=f"{coin_id}-ohlc-day.png", klass="day_plot")
                        line('p', f"Month Plot:", klass="bold_text")
                        doc.stag('img', src=f"{coin_id}-ohlc-month.png", klass="day_plot")
                    with tag('hr'):
                        text('')
    #  Save report
    report_file = open("screener-coins-report.html", "w")
    report_file.write(doc.getvalue())
    report_file.close()

In [31]:
#  Initialize clients
cg = CoinGeckoAPI()
#  Get all coingecko assets
max_coins = 5
month_ohlc_map = {}
info_map = {}
selected_coin_ids = []
coin_list = get_coin_ids()
num_selections = 0
max_selections = 3
i = 0
for coin in coin_list:
    coin_id = coin['id']
    name = coin['name']
    symbol = coin['symbol']
    #  Skip unwanted coins
    if "RealT" in name:
        continue
    print(f"Processing '{name}', ({symbol})")
#     #  Get monthly prices - 4 hour period
#     ohlc_df = get_ohlc(cg, coin_id, vs_currency='usd', days=30)
#     if len(ohlc_df) < 30:
#         continue
#     ohlc_df = calculate_technical_indicators(ohlc_df)
#     #  Calculate SMAs and RSI
#     metrics_df = calculate_metrics(ohlc_df)
#     selections_df = evaluate_conditions(metrics_df)
#     if len(selections_df) > 0:
#         #  Get coin info
    time.sleep(1.3)
    info_df = get_coin_info(cg, coin_id)
    if info_df is None:
        continue
    info_map[coin_id] = info_df
#         #  Store prices
#         ohlc_df.to_csv(f"{coin_id}_ohlc_df.csv")
#         print(f"-> Match: {coin_id}, {name}")
    selected_coin_ids.append(coin_id)
#         num_selections += 1
#         if num_selections >= max_selections:
#             break
    time.sleep(1.3)
#  Build report
build_report(selected_coin_ids, info_map)

Processing '01coin', (zoc)
Failed to fetch CoinGecko coin info 'coingecko_rank'
Processing 'Zus', (zcn)
Failed to fetch CoinGecko coin info 'coingecko_rank'
Processing '0 Knowledge Network', (0kn)
Failed to fetch CoinGecko coin info 'coingecko_rank'
Processing 'O-MEE', (ome)
Failed to fetch CoinGecko coin info 'coingecko_rank'
Processing '0VIX Protocol', (vix)


KeyboardInterrupt: 

In [32]:
build_report(selected_coin_ids, info_map)